In [3]:
import os
import re
import csv
import glob
import time
from collections import OrderedDict
import pandas as pd

## read Category:Women scientists (女性の科学者）articles data downloaded from PetScan (https://petscan.wmflabs.org/)
### User manual (https://meta.wikimedia.org/wiki/PetScan/)
-  read download.csv (Women by nationality|20 people|20 birth year|20)

In [16]:
df = pd.read_csv("female_scientist.csv")
df.head(20)

,number,title,pageid,namespace,length,touched
0,1,エイダ・ラブレス,1497,NaN,16304,20220308013829
1,2,アイン・ランド,16939,NaN,108623,20220117130533
2,3,ウィリアム・ハーシェル,18103,NaN,16884,20220219041044
3,4,ヒルデガルト・フォン・ビンゲン,29027,NaN,28469,20220529112034
4,5,アンゲラ・メルケル,29088,NaN,71960,20220606204400
5,6,フローレンス・ナイチンゲール,36834,NaN,29212,20220529000832
6,7,マリ・キュリー,38432,NaN,112129,20220506200151
7,8,ハンナ・アーレント,39270,NaN,42343,20220615031429
8,9,斎藤美津子,60488,NaN,10398,20200917021943
9,10,青木薫,61203,NaN,9797,20220503052635


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1363 entries, 0 to 1362
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   number     1363 non-null   int64  
 1   title      1363 non-null   object 
 2   pageid     1363 non-null   int64  
 3   namespace  0 non-null      float64
 4   length     1363 non-null   int64  
 5   touched    1363 non-null   int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 64.0+ KB


### In Denelezh (https://denelezh.wmcloud.org/gender-gap/?project=jawiki) there should be1,659 female scientists as in 2020-09-21


In [19]:
# key: folder's name, value: create dictionartris of max IDs
# exrtract max ID values from the last article in the last file in each directory
def mapping_dir(dir_path):
    dir_list = os.listdir(dir_path)
    for _dir in dir_list:
        last_f_path = sorted(glob.glob(os.path.join(dir_path, _dir) + '\\*'))[-1]
        max_id, _, _ = format_text(sorted(get_articlelist(last_f_path))[-1])
        dir_info[_dir] = max_id
    return dir_info


In [20]:
# pade_id from csv file and dir_info created by mapping_dir
# return folder name which include page_id I'm looking for
def get_search_dir(page_id, dir_info):
    min_id = 0
    for dir_name, max_id in dir_info.items():
        search_dir = dir_name
        if min_id <= page_id <= max_id:
            return search_dir
        else:
            min_id = max_id
    return next(iter(dir_info))     # if not found, retuen the first value(AA)
 

In [24]:
# If nothing is found with search_dir, search again from the beginning
# this can be replaced by "get_files"
def get_all_files(base_path, search_dir):
    dir_iter = iter(dir_info.keys())   # in case search_dir fails
    dir_list = list(dir_info.keys())   # list of key
    cnt = 1
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        search_dir = next(dir_iter)
        cnt += 1
        if search_dir == dir_list[-1] and cnt == len(dir_list):
            # when nothing found
            return

In [26]:
# 
def get_files(base_path, search_dir):
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        else:
            return

In [27]:
# return each article text
def format_text(data):
    head = head_reg.search(data)  # (<doc id=...>)
    id_tag = id_reg.search(head.group())
    doc_id = re.search(r'\d+', id_tag.group())  # extract id from header
    title_tag = title_reg.search(head.group())
    doc_title = re.search(r'".+"', title_tag.group())  
    doc_id = int(doc_id.group())
    title = doc_title.group().replace('"', '')      # delete ""
    # deete each header(<doc id=...>)and tag
    # extract only texts (delete title as some articles have duplicated tittles)
    text = data.replace(head.group(), '').rstrip('</doc>')[len(title):]
    return doc_id, title, text

In [28]:
# return lists of articles
def get_articlelist(f_path):
    f = open(f_path, 'r', encoding='utf-8')
    file_data = f.read()
    file_data = file_data.replace('\n', '')
    return doc_reg.findall(file_data)

In [29]:
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()

In [30]:
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()

In [31]:
if __name__ == '__main__':
    # 
    id_reg = re.compile(r'id="\d+"')
    title_reg = re.compile(r'title=".+"')
    doc_reg = re.compile(r'<doc id=.+?</doc>')
    head_reg = re.compile(r'<doc id.+?">')

In [32]:
# NEED TO CHANGE for differendt data
BASE_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor'
CSV_FILE = BASE_PATH + '/female_scientist.csv' # downloaded data from PetScan
WIKI_DIR = BASE_PATH + '/text'
OUTPUT_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor/f_scientist/'

In [33]:
# make dictionary of the max ID value in each folder
dir_info = OrderedDict()
dir_info = mapping_dir(WIKI_DIR)
counter = 1     # loop counter

In [34]:
# read csv
f = open(CSV_FILE, 'r', encoding='utf-8')
reader = csv.reader(f)
next(csv.reader(f))

['number', 'title', 'pageid', 'namespace', 'length', 'touched']

In [35]:
not_found = {}      # dict for not-found data

In [36]:
# measure time
start_time = time.time()

In [37]:
for line in reader:
        flag = False
        page_name = line[1]
        page_id = int(line[2])
        namespace = line[3]
        if namespace == "Category" or namespace == "Template":
            continue
        search_dir = get_search_dir(page_id, dir_info)
        for f_path in get_files(WIKI_DIR, search_dir):
            for d in get_articlelist(f_path):
                doc_id, title, text = format_text(d)
                if page_id == doc_id:
                    print(counter, title)
                    output_file(title, text)
                    counter += 1
                    flag = True
                    break
            if flag:
                break
        else:
            not_found[page_id] = page_name
            print("ページID: {}は見つかりませんでした.".format(page_id))
 
        end_time = time.time()
        processing_time = end_time - start_time
        print('processing_time(sec): ', processing_time)
 
        # print "not found" page and its ID
        for i, (_id, _title) in enumerate(not_found.items()):
            print(i, _id, _title)

F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
1 エイダ・ラブレス
processing_time(sec):  9.07904314994812
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
2 アイン・ランド
processing_time(sec):  11.186156511306763
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
3 ウィリアム・ハーシェル
processing_time(sec):  11.287999868392944
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
4 ヒルデガルト・フォン・ビンゲン
processing_time(sec):  12.494079351425171
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
5 アンゲラ・メルケル
processing_time(sec):  13.359483480453491
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
6 フローレンス・ナイチンゲール
processing_time(sec):  14.616837978363037
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
7 マリ・キュリー
processing_time(sec):  15.942381858825684
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
8 ハンナ・アーレント
processing_time(sec):  17.67474627494812
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
9 斎藤美津子
processing_time(sec):  19.12964963912964
F:/Wikiped

76 海保静子
processing_time(sec):  118.52929353713989
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AH
77 グレース・ホッパー
processing_time(sec):  119.06308817863464
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AH
78 ヘンリエッタ・スウォープ
processing_time(sec):  119.52236819267273
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AH
79 セシリア・ペイン＝ガポーシュキン
processing_time(sec):  119.96171450614929
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AH
80 ウィリアミーナ・フレミング
processing_time(sec):  120.41175198554993
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AH
81 浜矩子
processing_time(sec):  121.12893271446228
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AH
82 紺谷典子
processing_time(sec):  121.75557112693787
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AH
83 キャサリン・ブロジェット
processing_time(sec):  122.38733458518982
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AH
84 マーサ・ヌスバウム
processing_time(sec):  123.14670777320862
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AH


151 田口泰子
processing_time(sec):  254.20435905456543
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
152 シャーロット・アワーバック
processing_time(sec):  256.4423191547394
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
153 ジャネット・アクユズ・マッテイ
processing_time(sec):  258.8113820552826
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
154 マリア・ミッチェル
processing_time(sec):  260.9742224216461
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
155 澁谷知美
processing_time(sec):  264.0816740989685
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
156 丹下梅子
processing_time(sec):  266.3866047859192
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
157 今井通子
processing_time(sec):  268.44680070877075
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
158 金子康子
processing_time(sec):  270.72732853889465
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AL
159 リュドミーラ・チェルヌイフ
processing_time(sec):  271.04585337638855
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AL
160 エリザベ

227 針谷順子
processing_time(sec):  361.81594347953796
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
228 伊奈川愛菓
processing_time(sec):  363.2319359779358
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
229 マリア・ジビーラ・メーリアン
processing_time(sec):  364.94825053215027
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
230 松田文子
processing_time(sec):  366.5196125507355
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
231 青野篤子
processing_time(sec):  368.3050265312195
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
232 三宅幹子
processing_time(sec):  370.4405219554901
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
233 橋本優花里
processing_time(sec):  372.44911336898804
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
234 クリニック高田
processing_time(sec):  374.38525199890137
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
235 河野美代子
processing_time(sec):  376.52112579345703
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
236 加藤セチ
processing_time(

303 アリス・プリュヴォ＝フォル
processing_time(sec):  527.1843831539154
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AQ
304 藤川洋子
processing_time(sec):  528.3268809318542
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AQ
305 椹野道流
processing_time(sec):  529.7085800170898
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AQ
306 フローレンス・バスカム
processing_time(sec):  531.1376779079437
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AQ
307 エリヴィラ・ナビウリナ
processing_time(sec):  532.8195550441742
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AQ
308 小林みどり
processing_time(sec):  534.891496181488
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AQ
309 内田伸子
processing_time(sec):  537.1622669696808
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AQ
310 藤村逸子
processing_time(sec):  539.0440742969513
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AQ
311 タチアナ・ゴリコワ
processing_time(sec):  541.2608141899109
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AQ
312 友利新
processing

...

1057 山下富美代 (心理学者)
processing_time(sec):  1842.953541278839
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BE
1058 山田理恵子
processing_time(sec):  1843.378975391388
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BE
1059 山田里津
processing_time(sec):  1843.8736066818237
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BE
1060 望月享子
processing_time(sec):  1844.5878367424011
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BE
1061 ジョアン・スタビー
processing_time(sec):  1845.4397735595703
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BE
1062 津谷典子
processing_time(sec):  1846.8492410182953
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BE
1063 アントワネット・フーク
processing_time(sec):  1848.2903213500977
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BE
1064 クリスティン・ブラジー・フォード
processing_time(sec):  1849.356602191925
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BE
1065 フランシス・アーノルド
processing_time(sec):  1851.2288565635681
F:/Wikipedia/wikiextractor-master/wikiextractor

1131 マーリン・メルツァー
processing_time(sec):  1957.1261699199677
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
1132 松岡陽子 (計算機科学者)
processing_time(sec):  1957.9543585777283
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
1133 リザ・ギンズブルグ
processing_time(sec):  1958.7227416038513
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
1134 キャスリーン・アントネッリ
processing_time(sec):  1959.5648374557495
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
1135 エヴァ・フィルビン
processing_time(sec):  1960.4501960277557
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
1136 メアリー・ブラック (天文学者)
processing_time(sec):  1961.2990698814392
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
1137 アンナ・ウェッセルズ・ウィリアムズ
processing_time(sec):  1962.2583968639374
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
1138 ルーシー・シャピロ
processing_time(sec):  1963.2184314727783
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
1139 ファビオラ・ジャノッティ
processing_time(sec):  1964.1153557300568
F:/Wikipedia

1205 金山らく
processing_time(sec):  2067.2897956371307
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BG
1206 キャシー・シャーマズ
processing_time(sec):  2068.2993144989014
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BG
1207 エレノア・M・ハドレー
processing_time(sec):  2069.178436756134
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BG
1208 蟻川芳子
processing_time(sec):  2070.198008775711
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BG
1209 今市涼子
processing_time(sec):  2071.3058121204376
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BG
1210 武生房栄
processing_time(sec):  2072.648439884186
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BG
1211 アニー・イーズリー
processing_time(sec):  2073.8537056446075
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BG
1212 キャシー松井
processing_time(sec):  2074.9712929725647
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BG
1213 桂田芳枝
processing_time(sec):  2076.315088748932
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BG
1214 右田アサ
pr

1279 藤田弘子
processing_time(sec):  2196.9850549697876
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
1280 ジュディス・クロール
processing_time(sec):  2198.603499174118
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
1281 ボニー・ノートン
processing_time(sec):  2200.0846745967865
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
1282 キャサリーン・バルドヴィハリグ
processing_time(sec):  2201.7652826309204
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
1283 石田瑞穂 (科学者)
processing_time(sec):  2203.3136372566223
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
1284 アナ・キーゼンホファー
processing_time(sec):  2205.7448048591614
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
1285 ティティア・デ・ランゲ
processing_time(sec):  2208.5039978027344
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
1286 ボニー・バスラー
processing_time(sec):  2210.32661819458
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BH
1287 大隅正子
processing_time(sec):  2212.2007138729095
F:/Wikipedia/wikiextractor-master/wikiextr

1354 フランシス・ホーゲン
processing_time(sec):  2357.063053369522
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
1355 真田弘美
processing_time(sec):  2359.0882165431976
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
1356 マーガレット・マリー
processing_time(sec):  2360.5895829200745
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
1357 宇治梓紗
processing_time(sec):  2362.2061274051666
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
1358 シュネシオス
processing_time(sec):  2365.6857657432556
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
1359 跡見順子
processing_time(sec):  2367.4164204597473
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BI
1360 田中千穂子
processing_time(sec):  2369.6783463954926
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
ページID: 4605032は見つかりませんでした.
processing_time(sec):  2372.1003506183624
0 4605032 ナタリア・アヴシェンコ
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AA
ページID: 4605185は見つかりませんでした.
processing_time(sec):  2373.7415046691895
0 4605032 ナタリア